<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Lesson 25: Anomaly Detection</h1>

<a name='section_25_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L25.0 Overview</h2>


<h3>Navigation</h3>

<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_25_1">25.1 The Gaia Experiment</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_25_1">L25.1 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_25_2">25.2 Building Projections with Gaia</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_25_2">L25.2 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_25_3">25.3 Anomaly Detection with Gaia</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_25_3">L25.3 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_25_4">25.4 Anomaly Detection with Lots of Gaia Data</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_25_4">L25.4 Exercises</a></td>
    </tr>
</table>

In [ ]:
#>>>RUN: L25.0-runcell00

!git init
!git remote add -f origin https://github.com/mitx-8s50/nb_LEARNER/
!git config core.sparseCheckout true
!echo 'data/L25' >> .git/info/sparse-checkout
!git pull origin main


In [ ]:
#>>>RUN: L25.0-runcell01

!pip install astropy==6.0.0
!pip install astroquery==0.4.7
!pip install gala==1.8.1


In [ ]:
#>>>RUN: L25.0-runcell02

# astropy imports
import astropy.coordinates as coord
from astropy.table import QTable
import astropy.units as u
from astroquery.gaia import Gaia

# Third-party imports
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# gala imports
import gala.coordinates as gc
import gala.dynamics as gd
import gala.potential as gp
from gala.units import galactic

#ML imports
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.autograd import Variable


In [ ]:
#>>>RUN: L25.0-runcell03

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title

<a name='section_25_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C"> L25.1 The Gaia Experiment </h2>  

| [Top](#section_25_0) | [Previous Section](#section_25_0) | [Exercises](#exercises_25_1) | [Next Section](#section_25_2) |

In [ ]:
#>>>RUN: L25.1-runcell01

query_text = '''SELECT TOP 4096 ra, dec, parallax, pmra, pmdec, radial_velocity,
phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag
FROM gaiadr3.gaia_source
WHERE parallax_over_error > 10 AND
    parallax > 10 AND
    radial_velocity IS NOT null
ORDER BY random_index
'''

job = Gaia.launch_job(query_text)
gaia_data = job.get_results()
gaia_data.write('data/L25/gaia_data1.fits',overwrite=True)

#Note, if you return to this section after closing the kernel,
#you can load the data again using the following code
gaia_data = QTable.read('data/L25/gaia_data1.fits')


print("Total Events:",len(gaia_data))
print()
print(gaia_data[:4])

#Note, the columns in our dataset are defined by our query above.
#To print one column, use the command gaia_data['column_name'], e.g.: gaia_data['parallax']

In [ ]:
#>>>RUN: L25.1-runcell02

dist = coord.Distance(parallax=u.Quantity(gaia_data['parallax']))
print("Min:",dist.min(), "Max:",dist.max())
print(dist[0],1e3/gaia_data['parallax'][0])

plt.hist(dist)
plt.xlabel("distance (pcs)")
plt.ylabel("N")
plt.show()

In [ ]:
#>>>RUN: L25.1-runcell03

c = coord.SkyCoord(ra=gaia_data['ra'], dec=gaia_data['dec'],distance=dist,
                   pm_ra_cosdec=gaia_data['pmra'], pm_dec=gaia_data['pmdec'],
                   radial_velocity=gaia_data['radial_velocity'])

print(c[:4],'\n')
print()

#Now we can translate it to galactic coordinates
print(c.galactic[:4])
coord.Galactocentric()

In [ ]:
#>>>RUN: L25.1-runcell04

galcen = c.transform_to(coord.Galactocentric(z_sun=0*u.pc, galcen_distance=8.1*u.kpc))
print(galcen[:4],'\n')
plt.hist(galcen.z.value, bins=np.linspace(-110, 110, 32),alpha=0.5,label='z')
#plt.hist(galcen.x.value, bins=np.linspace(-110, 110, 32),alpha=0.5)
plt.hist(galcen.y.value, bins=np.linspace(-110, 110, 32),alpha=0.5,label='y')
plt.xlabel('Corrdinate position (about Galactic Plane) [{0:latex_inline}]'.format(galcen.z.unit));
plt.legend()
plt.show()

plt.hist(galcen.x.value, alpha=0.5)
plt.xlabel('X-position (nearby stars) [{0:latex_inline}]'.format(galcen.x.unit))
plt.legend()
plt.show()


In [ ]:
#>>>RUN: L25.1-runcell05

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.plot(galcen.v_x.value, galcen.v_y.value,marker='.', linestyle='none', alpha=0.5)
ax.set_xlim(-125, 125)
ax.set_ylim(200-125, 200+125)
ax.set_xlabel('vx [{0:latex_inline}]'.format(u.km/u.s))
ax.set_ylabel('vy [{0:latex_inline}]'.format(u.km/u.s))
plt.show()

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
plt.plot(galcen.v_x.value, galcen.v_z.value,marker='.', linestyle='none', alpha=0.5)
plt.xlabel('vx [{0:latex_inline}]'.format(u.km/u.s))
plt.ylabel('vz [{0:latex_inline}]'.format(u.km/u.s))
plt.xlim(-125, 125)
plt.ylim(-125, 125)
plt.show()


In [ ]:
#>>>RUN: L25.1-runcell06

M_G = gaia_data['phot_g_mean_mag'] - dist.distmod #corrected Star Magnitude (general-distance mod)
BP_RP = gaia_data['phot_bp_mean_mag'] - gaia_data['phot_rp_mean_mag'] #Blue filter - read filter

plt.plot(BP_RP.value, M_G.value, marker='.', linestyle='none', alpha=0.3)

plt.xlim(0, 3)
plt.ylim(11, 1)

#expand range
#plt.xlim(0, 3.75)
#plt.ylim(13.5, 0)

plt.xlabel('$G_{BP}-G_{RP}$')
plt.ylabel('$M_{G}$')
plt.show()

In [ ]:
#>>>RUN: L25.1-runcell07

np.seterr(invalid="ignore")
#Red+0.7 > Blue > Red+0.5 and 2 < Star magnitude < 3.75
hi_mass_mask = ((BP_RP > 0.5*u.mag) & (BP_RP < 0.7*u.mag) & (M_G > 2*u.mag) & (M_G < 3.75*u.mag))
#                &                 (np.abs(galcen.v_y - 220*u.km/u.s) < 50*u.km/u.s))

#Red+2.4 > Blue > Red+2 and 8.2 < Star magnitude < 9.7
lo_mass_mask = ((BP_RP > 2*u.mag) & (BP_RP < 2.4*u.mag) & (M_G > 8.2*u.mag) & (M_G < 9.7*u.mag))
#                &                (np.abs(galcen.v_y - 220*u.km/u.s) < 50*u.km/u.s))

hi_mass_color = 'tab:purple'
lo_mass_color = 'tab:red'
hi_mass_label = 'high mass'
lo_mass_label = 'low mass'
milky_way = gp.MilkyWayPotential()
milky_way

plt.plot(BP_RP.value, M_G.value, marker='.', linestyle='none', alpha=0.1)

for mask, color, label in zip([lo_mass_mask, hi_mass_mask],[lo_mass_color, hi_mass_color], [lo_mass_label, hi_mass_label]):
    plt.plot(BP_RP[mask].value, M_G[mask].value, marker='.', linestyle='none', 
            alpha=0.5, color=color, label=label)

plt.xlim(0, 3)
plt.ylim(11, 1)

plt.xlabel('$G_{BP}-G_{RP}$')
plt.ylabel('$M_{G}$')
plt.legend()
plt.show()
     
     

In [ ]:
#>>>RUN: L25.1-runcell08

fig, ax = plt.subplots(1, 1, figsize=(10, 10))

ax.plot(galcen[1==1].x.value, galcen[1==1].y.value,marker='.', linestyle='none', alpha=0.02,color='blue')
ax.plot(galcen[lo_mass_mask].x.value, galcen[lo_mass_mask].y.value,marker='.', linestyle='none', alpha=0.5,color=lo_mass_color, label=lo_mass_label)
ax.plot(galcen[hi_mass_mask].x.value, galcen[hi_mass_mask].y.value,marker='.', linestyle='none', alpha=0.5,color=hi_mass_color, label=hi_mass_label)
ax.set_xlabel('x [{0:latex_inline}]'.format(galcen.x.unit));
ax.set_ylabel('y [{0:latex_inline}]'.format(galcen.y.unit));
plt.legend()
plt.show()

In [ ]:
#>>>RUN: L25.1-runcell09

r_ce = np.sqrt(galcen[1==1]        .y.value**2 + galcen[1==1]        .z.value**2)
r_lo = np.sqrt(galcen[lo_mass_mask].y.value**2 + galcen[lo_mass_mask].z.value**2)
r_hi = np.sqrt(galcen[hi_mass_mask].y.value**2 + galcen[hi_mass_mask].z.value**2)

hist_ce, bin_edges = np.histogram(r_ce,bins=10, density=True)
hist_lo, bin_edges = np.histogram(r_lo, bins=bin_edges, density=True)
hist_hi, bin_edges = np.histogram(r_hi, bins=bin_edges, density=True)
bin_center = 0.5*(bin_edges[:-1] + bin_edges[1:])
sc=1./(len(galcen))
sl=1./(len(galcen[lo_mass_mask]))
sh=1./(len(galcen[hi_mass_mask]))
plt.errorbar(bin_center,hist_ce,yerr=sc*hist_ce**0.5,alpha=0.5,drawstyle="steps-mid",marker='.',color='blue',label='all')
plt.errorbar(bin_center,hist_lo,yerr=sl*hist_lo**0.5,alpha=0.5,drawstyle="steps-mid",marker='.',color=lo_mass_color,label=lo_mass_label)
plt.errorbar(bin_center,hist_hi,yerr=sh*hist_hi**0.5,alpha=0.5,drawstyle="steps-mid",marker='.',color=hi_mass_color,label=hi_mass_label)
plt.xlabel('y[pc]')
plt.legend()
plt.show()


vr_ce = np.sqrt(galcen[1==1]        .v_y.value**2 + galcen[1==1]        .v_x.value**2)
vr_lo = np.sqrt(galcen[lo_mass_mask].v_y.value**2 + galcen[lo_mass_mask].v_x.value**2)
vr_hi = np.sqrt(galcen[hi_mass_mask].v_y.value**2 + galcen[hi_mass_mask].v_x.value**2)

hist_ce, bin_edges = np.histogram(vr_ce,bins=20, density=True)
hist_lo, bin_edges = np.histogram(vr_lo, bins=bin_edges, density=True)
hist_hi, bin_edges = np.histogram(vr_hi, bins=bin_edges, density=True)
bin_center = 0.5*(bin_edges[:-1] + bin_edges[1:])
sc=1./(len(galcen))
sl=1./(len(galcen[lo_mass_mask]))
sh=1./(len(galcen[hi_mass_mask]))
plt.errorbar(bin_center,hist_ce,yerr=sc*hist_ce**0.5,alpha=0.5,drawstyle="steps-mid",marker='.',color='blue',label='all')
plt.errorbar(bin_center,hist_lo,yerr=sl*hist_lo**0.5,alpha=0.5,drawstyle="steps-mid",marker='.',color=lo_mass_color,label=lo_mass_label)
plt.errorbar(bin_center,hist_hi,yerr=sh*hist_hi**0.5,alpha=0.5,drawstyle="steps-mid",marker='.',color=hi_mass_color,label=hi_mass_label)
plt.xlabel('v$_{r}$[km/s]')
plt.legend()
plt.show()


vz_ce = galcen[1==1]        .v_z.value
vz_lo = galcen[lo_mass_mask].v_z.value
vz_hi = galcen[hi_mass_mask].v_z.value

hist_ce, bin_edges = np.histogram(vz_ce,bins=20, density=True)
hist_lo, bin_edges = np.histogram(vz_lo, bins=bin_edges, density=True)
hist_hi, bin_edges = np.histogram(vz_hi, bins=bin_edges, density=True)
bin_center = 0.5*(bin_edges[:-1] + bin_edges[1:])
sc=1./(len(galcen))
sl=1./(len(galcen[lo_mass_mask]))
sh=1./(len(galcen[hi_mass_mask]))
plt.errorbar(bin_center,hist_ce,yerr=sc*hist_ce**0.5,alpha=0.5,drawstyle="steps-mid",marker='.',color='blue',label='all')
plt.errorbar(bin_center,hist_lo,yerr=sl*hist_lo**0.5,alpha=0.5,drawstyle="steps-mid",marker='.',color=lo_mass_color,label=lo_mass_label)
plt.errorbar(bin_center,hist_hi,yerr=sh*hist_hi**0.5,alpha=0.5,drawstyle="steps-mid",marker='.',color=hi_mass_color,label=hi_mass_label)
plt.xlabel('v$_{z}$[km/s]')
plt.legend()
plt.show()


<a name='exercises_25_1'></a>     

| [Top](#section_25_0) | [Restart Section](#section_25_1) | [Next Section](#section_25_2) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 25.1.1</span>

We've compared the distributions of high mass stars and low mass stars. Now, partition the intermediate mass stars and compare their distributions.

Specifically, define two group of stars, `intermed1`, with a `BP_RP` color in the range `[1,1.5]` and a magnitude in the range `[5,7.5]`, and `intermed2`, with a `BP_RP` color in the range `[1.5,2]` and a magnitude in the range `[6.5,8.5]`.

Hint: It might be convenient to wrap some of the cells above into a function, which takes the masks, colors, and labels as inputs.


What do you see?

A) There appears to be no difference in the distributions.

B) These distributions are now clearly different.


<br>

In [ ]:
#>>>EXERCISE: L25.1.1

np.seterr(invalid="ignore")

#DEFINE MASKS
intermed1_mask = #YOUR CODE HERE
intermed2_mask = #YOUR CODE HERE

intermed1_color = 'tab:green'
intermed2_color = 'tab:orange'
intermed1_label = 'intermed1'
intermed2_label = 'intermed2'


#YOUR CODE HERE (APPLY MASKS, MAKE PLOTS)

     

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 25.1.2</span>

What if we consider some of the other variables? Select objects with extreme radial velocity, and see how they look in the HR diagram.  Specifically, select events with radial velocity < 100 km/s and compare with radial velocity > 290 km/s. Where do these objects appear on the HR diagram? Select ALL that apply.

A) The low velocity objects appear roughly evenly distributed in color and magnitude.
    
B) The low velocity objects appear to be bluer (younger) stars.

C) The low velocity objects appear to be redder (older) stars.

D) The high velocity objects appear roughly evenly distributed in color and magnitude.

E) The high velocity objects appear to be bluer (younger) stars.

F) The high velocity objects appear to be redder (older) stars.


<br>

In [ ]:
#>>>EXERCISE: L25.1.2

#DEFINE MASKS
np.seterr(invalid="ignore")

lo_vel_mask = #YOUR CODE HERE
hi_vel_mask = #YOUR CODE HERE

hi_vel_color = 'tab:purple'
lo_vel_color = 'tab:orange'
hi_vel_label = 'high velocity'
lo_vel_label = 'low velocity'

#MAKE PLOT OF HR DIAGRAM
#suggestion: use larger marker style or size to see clearly

     

<a name='section_25_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C"> L25.2 Building Projections with Gaia </h2>  

| [Top](#section_25_0) | [Previous Section](#section_25_1) | [Exercises](#exercises_25_2) | [Next Section](#section_25_3) |

In [ ]:
#>>>RUN: L25.2-runcell01

H = gp.Hamiltonian(milky_way)
w0_hi = gd.PhaseSpacePosition(galcen[hi_mass_mask].cartesian)
w0_lo = gd.PhaseSpacePosition(galcen[lo_mass_mask].cartesian)

orbits_hi = H.integrate_orbit(w0_hi, dt=1*u.Myr, t1=0*u.Myr, t2=500*u.Myr)
orbits_lo = H.integrate_orbit(w0_lo, dt=1*u.Myr, t1=0*u.Myr, t2=500*u.Myr)

w0_hlo,bin_edges = np.histogram(w0_lo.z.value,bins=10,density=True)
w0_hhi,bin_edges = np.histogram(w0_hi.z.value,bins=bin_edges,density=True)
bin_center = 0.5*(bin_edges[:-1] + bin_edges[1:])

print(orbits_lo[-1,0],w0_lo[0])
o0_hlo,bin_edges = np.histogram(1e3*orbits_lo[-1,:].z.value,bins=bin_edges,density=True)
o0_hhi,bin_edges = np.histogram(1e3*orbits_hi[-1,:].z.value,bins=bin_edges,density=True)

plt.plot(bin_center,w0_hlo,alpha=0.5,marker='.',drawstyle="steps-mid",label='z-start low mass')
plt.plot(bin_center,w0_hhi,alpha=0.5,marker='.',drawstyle="steps-mid",label='z-start high mass')

plt.plot(bin_center,o0_hlo,alpha=0.5,marker='.',drawstyle="steps-mid",label='z-end low mass')
plt.plot(bin_center,o0_hhi,alpha=0.5,marker='.',drawstyle="steps-mid",label='z-end high mass')
plt.legend()
plt.xlabel('z[pc]')
plt.ylabel('N')

plt.show()

In [ ]:
#>>>RUN: L25.2-runcell02

fig = orbits_hi[0:500, 0].plot(color=hi_mass_color)
_   = orbits_lo[0:500, 0].plot(axes=fig.axes, color=lo_mass_color)

In [ ]:
#>>>RUN: L25.2-runcell03

plt.plot(1e-3*galcen[1==1].x.value, 1e-3*galcen[1==1].y.value,marker='.', linestyle='none', alpha=0.2,color='blue')
plt.plot(1e-3*w0_lo.x.value,1e-3*w0_lo.y.value,marker='.', linestyle='none', alpha=0.5,color='green',label='start')
plt.plot(orbits_lo[-1].x.value,orbits_lo[-1].y.value,marker='.', linestyle='none', alpha=0.5,color=lo_mass_color,label='low mass')
plt.plot(orbits_hi[-1].x.value,orbits_hi[-1].y.value,marker='.', linestyle='none', alpha=0.5,color=hi_mass_color,label='high mass')
plt.xlabel('x[kpc]')
plt.ylabel('y[kpc]')
plt.legend()
plt.show()

plt.plot(1e-3*galcen[1==1].z.value, 1e-3*galcen[1==1].y.value,marker='.', linestyle='none', alpha=0.2,color='blue')#,label='origin')
plt.plot(1e-3*w0_lo.z.value,1e-3*w0_lo.y.value,marker='.', linestyle='none', alpha=0.5,color='green',label='start')
plt.plot(orbits_lo[-1].z.value,orbits_lo[-1].y.value,marker='.', linestyle='none', alpha=0.5,color=lo_mass_color,label='low mass')
plt.plot(orbits_hi[-1].z.value,orbits_hi[-1].y.value,marker='.', linestyle='none', alpha=0.5,color=hi_mass_color,label='high mass')
plt.xlabel('z[kpc]')
plt.ylabel('y[kpc]')
plt.legend()
plt.show()

In [ ]:
#>>>RUN: L25.2-runcell04

fig = orbits_hi[:, 0].cylindrical.plot(['rho', 'z'], 
                                       color=hi_mass_color,
                                       label='high mass')
_ = orbits_lo[:, 0].cylindrical.plot(['rho', 'z'], color=lo_mass_color,
                                     axes=fig.axes,
                                     label='low mass')

fig.axes[0].legend(loc='upper left')
fig.axes[0].set_ylim(-0.3, 0.3)
plt.show()

plt.plot(1e-3*galcen[1==1].cylindrical.rho, 1e-3*galcen[1==1].z.value,marker='.', linestyle='none', alpha=0.2,color='blue')
plt.plot(1e-3*w0_lo.cylindrical.rho,        1e-3*w0_lo.z.value,marker='.', linestyle='none', alpha=0.5,color='green')
plt.plot(orbits_lo[-1].cylindrical.rho,orbits_lo[-1].z.value,marker='.', linestyle='none', alpha=0.5,color=lo_mass_color)
plt.plot(orbits_hi[-1].cylindrical.rho,orbits_hi[-1].z.value,marker='.', linestyle='none', alpha=0.5,color=hi_mass_color)
plt.xlabel(r'$\rho$ [kpc]')
plt.ylabel('z [kpc]')
plt.show()

In [ ]:
#>>>RUN: L25.2-runcell05

zmax_hi = orbits_hi.zmax(approximate=True)
zmax_lo = orbits_lo.zmax(approximate=True)
bins = np.linspace(0, 2, 50)

plt.hist(zmax_hi.value, bins=bins, alpha=0.4, density=True, label='high-mass', color=hi_mass_color)
plt.hist(zmax_lo.value, bins=bins, alpha=0.4, density=True, label='low-mass',color=lo_mass_color);
plt.legend(loc='best', fontsize=14)
print("Mean high: ", zmax_hi.value.mean(),"Mean Low:",zmax_lo.mean())

plt.yscale('log')
plt.xlabel(r" zmax" + " [{0:latex}]".format(zmax_hi.unit))
plt.show()

zmax_hi = orbits_hi.eccentricity()
zmax_lo = orbits_lo.eccentricity()
print("Ecc high: ", zmax_hi.value.mean(),"Ecc Low:",zmax_lo.mean())

bins = np.linspace(0, 2, 50) #bins = np.linspace(0, 0.75, 50)
plt.hist(zmax_hi.value, bins=bins, alpha=0.4, density=True, label='high-mass', color=hi_mass_color)
plt.hist(zmax_lo.value, bins=bins, alpha=0.4, density=True, label='low-mass',color=lo_mass_color);
plt.legend(loc='best', fontsize=14)
plt.yscale('log')
plt.xlabel('Eccentricity')
plt.show()

<a name='exercises_25_2'></a>     

| [Top](#section_25_0) | [Restart Section](#section_25_2) | [Next Section](#section_25_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 25.2.1</span>

Let's make a histogram of another characteristic, namely the orbital period. Plot the distribution of orbital periods for these populations of stars. Note you will need the `estimate_period(radial=True)` function from <a href="https://gala.adrian.pw/en/latest/api/gala.dynamics.Orbit.html#gala.dynamics.Orbit.estimate_period" target="_blank">here</a>. 

As a rough measurement of how regular the periods are for each group, calculate the standard deviation of the orbital periods for the high-mass vs. low-mass stars. Report your answer as a list of numbers with `[per_hi_stdev,per_lo_stdev]`, with precision 1 Myr.


<br>

In [ ]:
#>>>EXERCISE: L25.2.1

per_hi = #YOUR CODE HERE
per_lo = #YOUR CODE HERE

#CALCULATE THE STDEV OF EACH
#YOUR CODE HERE

#PLOT THE HISTOGRAM
bins = np.linspace(100, 250, 50)
plt.hist(per_hi.value, bins=bins, alpha=0.4, density=True, label='high-mass', color=hi_mass_color)
plt.hist(per_lo.value, bins=bins, alpha=0.4, density=True, label='low-mass',color=lo_mass_color);
plt.legend(loc='best', fontsize=14)
plt.yscale('log')
plt.xlabel('period')
plt.show()

<a name='section_25_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C"> L25.3 Anomaly Detection with Gaia </h2>  

| [Top](#section_25_0) | [Previous Section](#section_25_2) | [Exercises](#exercises_25_3) | [Next Section](#section_25_4) |

In [ ]:
#>>>RUN: L25.3-runcell01

def prepData(igaia_data,idist,igalcen,iSplit=0.5):
    gaia_vars=['phot_g_mean_mag','phot_bp_mean_mag','phot_rp_mean_mag']
    var0=(igaia_data[gaia_vars[0]]-idist.distmod).value
    var1=(igaia_data[gaia_vars[1]]-idist.distmod).value
    var2=(igaia_data[gaia_vars[2]]-idist.distmod).value
    var3=igalcen.x.value
    var4=igalcen.y.value
    var5=igalcen.z.value
    var6=igalcen.v_x.value
    var7=igalcen.v_y.value
    var8=igalcen.v_z.value

    #processed_data = np.vstack((var0,var1,var2,var3,var4,var5,var6,var7,var8))
    processed_data = np.vstack((var0,var1,var2,var3,var4,var5,var6,var7,var8))
    processed_data = processed_data.T
    processed_data = processed_data[~np.isnan(processed_data).any(axis=1)]
    processed_data_raw = processed_data.copy()

    #normalize the data
    processed_data /= np.std(processed_data,axis=0)
    processed_data -= np.mean(processed_data,axis=0)
    processed_data = processed_data[~np.isnan(processed_data).any(axis=1)]
    
    #pytorch the layer
    tprocessed_data = torch.tensor(processed_data).float()
    processed_data_raw = processed_data_raw[~torch.any(tprocessed_data.isnan(),dim=1)]
    galcen_clean       = igalcen[~torch.any(tprocessed_data.isnan(),dim=1)]
    #split
    maxindex       = int(len(processed_data)*iSplit)
    trainset       = torch.tensor(processed_data[0:maxindex]).float()
    trainset       = trainset[~torch.any(trainset.isnan(),dim=1)]
    testset        = torch.tensor(processed_data[maxindex:len(processed_data)]).float()
    testset        = testset[~torch.any(testset.isnan(),dim=1)]
    print(processed_data_raw.shape,testset.shape,trainset.shape)
    return testset,trainset,processed_data_raw,tprocessed_data,galcen_clean

btestset,btrainset,bprocessed_data_raw,btprocessed_data,bgalcen_clean=prepData(gaia_data,dist,galcen)

In [ ]:
#>>>RUN: L25.3-runcell02

class MLP(nn.Module):
    def __init__(self,n_inputs,n_outputs):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(n_inputs, 20),
            nn.ReLU(),
            nn.Linear(20, 6),
            nn.ReLU(),
            nn.Linear(6, 2),
            nn.ReLU(),
            nn.Linear(2, 6),
            nn.ReLU(),
            nn.Linear(6, 20),
            nn.ReLU(),
            nn.Linear(20, n_outputs),
        )
        
    def forward(self, x):        
        x = self.layers(x)
        return x

def train(x,y,net,loss_func,opt,sched,nepochs):
    net.train(True)
    for epoch in range(nepochs):
        prediction = net(x)
        opt.zero_grad()
        loss = loss_func(prediction,y) 
        loss.backward() 
        opt.step()
        if epoch % 500 == 0: 
            print('[%d] loss: %.4f ' % (epoch + 1, loss.item()  ))
    #sched.step()
    return    

basicmodel     = MLP(btrainset.shape[1],btrainset.shape[1])
optimizer = torch.optim.Adam(basicmodel.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.1, last_epoch=-1, verbose=False)
loss_fn   =  nn.MSELoss(reduction='sum')

In [ ]:
#>>>RUN: L25.3-runcell03

train(btrainset,btrainset,basicmodel,loss_fn,optimizer,scheduler,10001)

In [ ]:
#>>>RUN: L25.3-runcell04

basicmodel.train(False)
boutput=basicmodel(btestset)
btestloss=torch.sum((btestset-boutput)**2,axis=1)
plt.hist(btestloss[btestloss < 100].detach().numpy(),density=True)
plt.yscale('log')
plt.xlabel('loss')
plt.ylabel('pdf')
plt.show()

varlabels=['Mag','B-Mag','R-Mag','x','y','z','vx','vy','vz']

fig, ax = plt.subplots(3, 3, figsize=(20, 20))
for var in range(btestset.shape[1]):
    _,bins,_=ax[var//3,var % 3].hist(btestset[:,var].detach().numpy(),density=True,alpha=0.5,label='Input')
    ax[var//3,var % 3].hist(boutput [:,var].detach().numpy(),density=True,alpha=0.5,bins=bins,label='Output')
    ax[var//3,var % 3].set_xlabel(varlabels[var])
    ax[var//3,var % 3].legend()

In [ ]:
#>>>RUN: L25.3-runcell05

def plotAnomaly(iCut,iRaw,iLoss,igalcen,maxlosscolor=20):
    loss = np.minimum(iLoss,maxlosscolor)
    anomalies=(iLoss > iCut)
    baseidex=len(iRaw)-len(iLoss)
    btestdata_raw = iRaw[baseidex:]
    anomaly_raw  = btestdata_raw[anomalies]
    btestgalcen   = igalcen[baseidex:]
    anomaly_galcen = btestgalcen[anomalies]
    loss = loss/np.max(loss)
    if maxlosscolor != 20:
        loss=np.ones(loss.shape)
    print(loss.shape,btestdata_raw[:,2].shape)
    scat=plt.scatter(btestdata_raw[:,2]-btestdata_raw[:,1],btestdata_raw[:,0], marker='.',c=loss,cmap="viridis")
    plt.plot(anomaly_raw[:,2]-anomaly_raw[:,1],anomaly_raw[:,0], marker='.', linestyle='none',c='orange')
    plt.xlabel('$G_{BP}-G_{RP}$')
    plt.ylabel('$M_{G}$')
    plt.ylim(15,0)
    plt.xlim(1,-5)
    plt.colorbar(scat)
    plt.show()
    
    scat=plt.scatter(btestdata_raw[:,3],btestdata_raw[:,4],c=loss, marker='.',cmap="viridis")
    plt.plot(anomaly_raw[:,3],anomaly_raw[:,4], marker='.', linestyle='none',c='orange')
    plt.xlabel("x[pc]")
    plt.ylabel("y[pc]")
    plt.colorbar(scat)
    plt.show()

    scat=plt.scatter(btestdata_raw[:,3],btestdata_raw[:,5],c=loss, marker='.', cmap="viridis")
    plt.plot(anomaly_raw[:,3],anomaly_raw[:,5], marker='.', linestyle='none',c='orange')
    plt.xlabel("x[pc]")
    plt.ylabel("z[pc]")
    plt.colorbar(scat)
    plt.show()

    scat=plt.scatter(btestdata_raw[:,6],btestdata_raw[:,7],c=loss, marker='.', cmap="viridis")
    plt.plot(anomaly_raw[:,6],anomaly_raw[:,7], marker='.', linestyle='none',c='orange')
    plt.xlabel("vx[pc]")
    plt.ylabel("vy[pc]")
    plt.colorbar(scat)
    plt.show()

    scat=plt.scatter(btestdata_raw[:,6],btestdata_raw[:,8],c=loss, marker='.',cmap="viridis")
    plt.plot(anomaly_raw[:,6],anomaly_raw[:,8], marker='.', linestyle='none')
    plt.xlabel("vx[pc]")
    plt.ylabel("vz[pc]")
    plt.show()

    print(len(btestdata_raw),len(iRaw),len(igalcen),len(btestgalcen))
    H = gp.Hamiltonian(milky_way)
    w0_anom = gd.PhaseSpacePosition(anomaly_galcen.cartesian)
    orbits_anom = H.integrate_orbit(w0_anom, dt=1*u.Myr, t1=0*u.Myr, t2=100*u.Myr)
    w0_all  = gd.PhaseSpacePosition(bgalcen_clean[1==1].cartesian)
    orbits_all  = H.integrate_orbit(w0_all,  dt=1*u.Myr, t1=0*u.Myr, t2=100*u.Myr)

    zmax_all = orbits_all.zmax(approximate=True)
    zmax_anom = orbits_anom.zmax(approximate=True)
    print("ZMax mean:",zmax_anom[~np.isnan(zmax_anom.value)].mean(),len(zmax_anom),"default:",zmax_all.mean())
    bins = np.linspace(0, 10, 50)
    plt.hist(zmax_all.value, bins=bins, alpha=0.4, density=True, label='all')
    plt.hist(zmax_anom.value, bins=bins, alpha=0.4, density=True, label='anom')
    plt.legend(loc='best', fontsize=14)
    plt.yscale('log')
    plt.xlabel(r" zmax" + " [{0:latex}]".format(zmax_all.unit))
    plt.show()

    zmax_all  = orbits_all.eccentricity()
    zmax_anom = orbits_anom.eccentricity()
    print("Ecc mean:",zmax_anom[~np.isnan(zmax_anom.value)].mean(),len(zmax_anom),"default:",zmax_all.mean())
    bins = np.linspace(0, 3, 50)
    plt.hist(zmax_all.value,  bins=bins, alpha=0.4, density=True, label='all')
    plt.hist(zmax_anom.value, bins=bins, alpha=0.4, density=True, label='anom')
    plt.legend(loc='best', fontsize=14)
    plt.yscale('log')
    plt.xlabel('Eccentricity')
    plt.show()


plotAnomaly(10,bprocessed_data_raw,btestloss.detach().numpy(),bgalcen_clean)

<a name='exercises_25_3'></a>     

| [Top](#section_25_0) | [Restart Section](#section_25_3) | [Next Section](#section_25_4) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 25.3.1</span>

We saw that anomalous stars tended to have large $|v_{x}|$ values. Let's try to cut on the events with $|v_{x}| > 100$ and thereby effectively reverse engineer the anomaly detection. 

In the function `plotAnomaly(iCut,iRaw,iLoss,igalcen,maxlosscolor=20)`, the anomalous objects have `iLoss` values that are above the threshold defined by `iCut`. So to accomplish our goal, let's define an array of `iLoss` values to be the velocities  $|v_{x}|$, then use the appropriate `iCut`.

Consider the options below for defining this cut on the data, and choose the best one. Hint, look at the definitions within the `prepData` function at the beginning of this section.

A) `cutvals=np.abs(bprocessed_data_raw[:,0][2032:]).copy()`\
B) `cutvals=np.abs(bprocessed_data_raw[:,1][2032:]).copy()`\
C) `cutvals=np.abs(bprocessed_data_raw[:,2][2032:]).copy()`\
D) `cutvals=np.abs(bprocessed_data_raw[:,3][2032:]).copy()`\
E) `cutvals=np.abs(bprocessed_data_raw[:,4][2032:]).copy()`\
F) `cutvals=np.abs(bprocessed_data_raw[:,5][2032:]).copy()`\
G) `cutvals=np.abs(bprocessed_data_raw[:,6][2032:]).copy()`\
H) `cutvals=np.abs(bprocessed_data_raw[:,7][2032:]).copy()`\
I) `cutvals=np.abs(bprocessed_data_raw[:,8][2032:]).copy()`


Upon completing your selection above, try to run the code `plotAnomaly(100,bprocessed_data_raw,cutvals,bgalcen_clean,maxlosscolor=150)` and compare the HR diagram to what we saw before. Do you think this does a good job?

<br>

In [ ]:
#>>>EXERCISE: L25.3.1

cutvals=#YOUR CODE HERE
plotAnomaly(100,bprocessed_data_raw,cutvals,bgalcen_clean,maxlosscolor=150)

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 25.3.2</span>

Search for anomalies without using the red, blue, and general filters. Hint, to do this you could either redfine the `prepData` function, or perform the relavant selections on the existing data (e.g., `btestset[:,3:9]`). How do things compare? Select ALL that apply.

A) The selected anomalies are identical.

B) The selected anomalies are totally different.

C) Some anomalies are the same, but the selection is different from before.

D) The loss function is nearly identical.

E) The loss function has a narrower tail, so fewer anomalies are selected.

F) The loss function has a wider tail, so more anomalies are selected.

<br>

<a name='section_25_4'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C"> L25.4 Anomaly Detection with Lots of Gaia Data</h2>  

| [Top](#section_25_0) | [Previous Section](#section_25_3) | [Exercises](#exercises_25_4) |

In [ ]:
#>>>RUN: L25.4-runcell01

#note, our query looks for 1e6 stars, but only finds about 170k
query_text = '''SELECT TOP 1000000 ra, dec, parallax, pmra, pmdec, radial_velocity,
phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag
FROM gaiadr3.gaia_source
WHERE parallax_over_error > 10 AND
    parallax > 10 AND
    radial_velocity IS NOT null
ORDER BY random_index
'''

job = Gaia.launch_job(query_text)
gaia_data = job.get_results()
gaia_data.write('data/L25/gaia_data2.fits',overwrite=True)

#Note, if you return to this section after closing the kernel,
#you can load the data again using the following code
gaia_data = QTable.read('data/L25/gaia_data2.fits')
print("Total Events:",len(gaia_data))


In [ ]:
#>>>RUN: L25.4-runcell02

dist = coord.Distance(parallax=u.Quantity(gaia_data['parallax']))
M_G = gaia_data['phot_g_mean_mag'] - dist.distmod
BP_RP = gaia_data['phot_bp_mean_mag'] - gaia_data['phot_rp_mean_mag']

plt.plot(BP_RP.value, M_G.value, marker='.', linestyle='none', alpha=0.3)

plt.xlim(-1, 5)
plt.ylim(17, -2)

plt.xlabel('$G_{BP}-G_{RP}$')
plt.ylabel('$M_{G}$')
plt.show()

In [ ]:
#>>>RUN: L25.4-runcell03

#redefine as above
def prepData(igaia_data,idist,igalcen,iSplit=0.5):
    gaia_vars=['phot_g_mean_mag','phot_bp_mean_mag','phot_rp_mean_mag']
    var0=(igaia_data[gaia_vars[0]]-idist.distmod).value
    var1=(igaia_data[gaia_vars[1]]-idist.distmod).value
    var2=(igaia_data[gaia_vars[2]]-idist.distmod).value
    var3=igalcen.x.value
    var4=igalcen.y.value
    var5=igalcen.z.value
    var6=igalcen.v_x.value
    var7=igalcen.v_y.value
    var8=igalcen.v_z.value

    #processed_data = np.vstack((var0,var1,var2,var3,var4,var5,var6,var7,var8))
    processed_data = np.vstack((var0,var1,var2,var3,var4,var5,var6,var7,var8))
    processed_data = processed_data.T
    processed_data = processed_data[~np.isnan(processed_data).any(axis=1)]
    processed_data_raw = processed_data.copy()

    #normalize the data
    processed_data /= np.std(processed_data,axis=0)
    processed_data -= np.mean(processed_data,axis=0)
    processed_data = processed_data[~np.isnan(processed_data).any(axis=1)]
    
    #pytorch the layer
    tprocessed_data = torch.tensor(processed_data).float()
    processed_data_raw = processed_data_raw[~torch.any(tprocessed_data.isnan(),dim=1)]
    galcen_clean       = igalcen[~torch.any(tprocessed_data.isnan(),dim=1)]
    #split
    maxindex       = int(len(processed_data)*iSplit)
    trainset       = torch.tensor(processed_data[0:maxindex]).float()
    trainset       = trainset[~torch.any(trainset.isnan(),dim=1)]
    testset        = torch.tensor(processed_data[maxindex:len(processed_data)]).float()
    testset        = testset[~torch.any(testset.isnan(),dim=1)]
    print(processed_data_raw.shape,testset.shape,trainset.shape)
    return testset,trainset,processed_data_raw,tprocessed_data,galcen_clean

c = coord.SkyCoord(ra=gaia_data['ra'], dec=gaia_data['dec'],distance=dist,pm_ra_cosdec=gaia_data['pmra'], pm_dec=gaia_data['pmdec'],
                   radial_velocity=gaia_data['radial_velocity'])
galcen = c.transform_to(coord.Galactocentric(z_sun=0*u.pc, galcen_distance=8.1*u.kpc))

testset,trainset,processed_data_raw,tprocessed_data,galcen_clean=prepData(gaia_data,dist,galcen)

In [ ]:
#>>>RUN: L25.4-runcell04

class MLP(nn.Module):
    def __init__(self,n_inputs,n_outputs):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(n_inputs, 100),
            nn.ReLU(),
            nn.Linear(100, 20),
            nn.ReLU(),
            nn.Linear(20, 4),
            nn.ReLU(),
            nn.Linear(4, 20),
            nn.ReLU(),
            nn.Linear(20, 100),
            nn.ReLU(),
            nn.Linear(100, n_outputs),
        )
        
    def forward(self, x):        
        x = self.layers(x)
        return x

def train(x,y,net,loss_func,opt,sched,nepochs):
    net.train(True)
    for epoch in range(nepochs):
        prediction = net(x)
        opt.zero_grad()
        loss = loss_func(prediction,y) 
        loss.backward() 
        opt.step()
        if epoch % 500 == 0: 
            print('[%d] loss: %.4f ' % (epoch + 1, loss.item()  ))
    #sched.step()
    return    

model     = MLP(trainset.shape[1],trainset.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.1, last_epoch=-1, verbose=False)
loss_fn   =  nn.MSELoss(reduction='sum')

In [ ]:
#>>>RUN: L25.4-runcell05

train(trainset,trainset,model,loss_fn,optimizer,scheduler,10001)
#train(trainset,trainset,model,loss_fn,optimizer,scheduler,5001) #train for shorter amount of epochs

In [ ]:
#>>>RUN: L25.4-runcell06

model.train(False)
output=model(testset)
testloss=torch.sum((testset-output)**2,axis=1)
plt.hist(testloss[testloss < 100].detach().numpy(),density=True)
plt.yscale('log')
plt.xlabel('loss')
plt.ylabel('pdf')
plt.show()

varlabels=['Mag','B-Mag','R-Mag','x','y','z','vx','vy','vz']

fig, ax = plt.subplots(3, 3, figsize=(20, 20))
for var in range(testset.shape[1]):
    _,bins,_=ax[var//3,var % 3].hist(testset[:,var].detach().numpy(),density=True,alpha=0.5,label='Input')
    ax[var//3,var % 3].hist(output [:,var].detach().numpy(),density=True,alpha=0.5,bins=bins,label='Output')
    ax[var//3,var % 3].set_xlabel(varlabels[var])
    ax[var//3,var % 3].legend()

In [ ]:
#>>>RUN: L25.4-runcell07

milky_way = gp.MilkyWayPotential()
H = gp.Hamiltonian(milky_way)
w0_all  = gd.PhaseSpacePosition(galcen_clean[1==1].cartesian)
orbits_all  = H.integrate_orbit(w0_all,  dt=1*u.Myr, t1=0*u.Myr, t2=100*u.Myr)
#plotAnomaly(45,processed_data_raw,testloss,galcen_clean,orbits_all)

In [ ]:
#>>>RUN: L25.4-runcell08

anomalies=(testloss > 45.)
baseidex=len(processed_data_raw)-len(testloss)
testgalcen   = galcen_clean[baseidex:]
anomaly_galcen = testgalcen[anomalies]

w0_anom = gd.PhaseSpacePosition(anomaly_galcen.cartesian)
orbits_anom = H.integrate_orbit(w0_anom, dt=1*u.Myr, t1=0*u.Myr, t2=100*u.Myr)

#redefine, as above
hi_mass_color = 'tab:purple'
lo_mass_color = 'tab:red'
hi_mass_label = 'high mass'
lo_mass_label = 'low mass'

fig = orbits_all[0:200, 0].plot(color=hi_mass_color,label='normal')
_   = orbits_all[0:200, 1].plot(axes=fig.axes,color=hi_mass_color)
_   = orbits_all[0:200, 2].plot(axes=fig.axes,color=hi_mass_color)
_   = orbits_anom[0:200, 0].plot(axes=fig.axes,label='anom 1')
_   = orbits_anom[0:200, 1].plot(axes=fig.axes,label='anom 2')
_   = orbits_anom[0:200, 2].plot(axes=fig.axes,label='anom 3')
_   = orbits_anom[0:200, 3].plot(axes=fig.axes,label='anom 4',color=lo_mass_color)
plt.legend()


zmax=orbits_anom.zmax(approximate=True)
print(zmax)

In [ ]:
#>>>RUN: L25.4-runcell09

def plotAnomalyBasic(iCut,iRaw,iLoss,igalcen,iOrbitsAll):
    loss = np.minimum(iLoss.detach().numpy(),20)
    anomalies=(iLoss > iCut)
    baseidex=len(iRaw)-len(iLoss)
    testdata_raw = iRaw[baseidex:]
    anomaly_raw  = testdata_raw[anomalies]
    testgalcen   = igalcen[baseidex:]
    anomaly_galcen = testgalcen[anomalies]

    scat=plt.scatter(-1.*(testdata_raw[:,2]-testdata_raw[:,1]),-1.*testdata_raw[:,0],c=loss)
    plt.plot(-1.*(anomaly_raw[:,2]-anomaly_raw[:,1]),-1.*anomaly_raw[:,0], marker='.',c='orange', linestyle='none')
    plt.xlabel('$G_{BP}-G_{RP}$')
    plt.ylabel('$M_{G}$')
    plt.colorbar(scat)
    plt.show()

    scat=plt.scatter(testdata_raw[:,3],testdata_raw[:,4], marker='.', c=loss)
    plt.plot(anomaly_raw[:,3],anomaly_raw[:,4], marker='.',c='orange', linestyle='none')
    plt.xlabel("x[pc]")
    plt.ylabel("y[pc]")
    plt.colorbar(scat)
    plt.show()

    scat=plt.scatter(testdata_raw[:,3],testdata_raw[:,5], marker='.', c=loss)
    plt.plot(anomaly_raw[:,3],anomaly_raw[:,5], marker='.',c='orange', linestyle='none')
    plt.xlabel("x[pc]")
    plt.ylabel("z[pc]")
    plt.colorbar(scat)
    plt.show()

    scat=plt.scatter(testdata_raw[:,6],testdata_raw[:,7], marker='.', c=loss)
    plt.plot(anomaly_raw[:,6],anomaly_raw[:,7], marker='.',c='orange', linestyle='none')
    plt.xlabel("vx[pc]")
    plt.ylabel("vy[pc]")
    plt.colorbar(scat)
    plt.show()

    scat=plt.scatter(testdata_raw[:,6],testdata_raw[:,8], marker='.', c=loss)
    plt.plot(anomaly_raw[:,6],anomaly_raw[:,8], marker='.',c='orange', linestyle='none')
    plt.xlabel("vx[pc]")
    plt.ylabel("vz[pc]")
    plt.colorbar(scat)
    plt.show()

def plotAnomalyComplex(iCut,iRaw,iLoss,igalcen,iOrbitsAll,iEccAll,iZAll,iT2):
    anomalies=(iLoss > iCut)
    baseidex=len(iRaw)-len(iLoss)
    testdata_raw = iRaw[baseidex:]
    anomaly_raw  = testdata_raw[anomalies]
    testgalcen   = igalcen[baseidex:]
    anomaly_galcen = testgalcen[anomalies]

    print(len(testdata_raw),len(iRaw),len(igalcen),len(testgalcen))
    H = gp.Hamiltonian(milky_way)
    w0_anom = gd.PhaseSpacePosition(anomaly_galcen.cartesian)
    orbits_anom = H.integrate_orbit(w0_anom, dt=1*u.Myr, t1=0*u.Myr, t2=iT2)

    #zmax_all = iOrbitsAll.zmax(approximate=True)
    zmax_anom = orbits_anom.zmax(approximate=True)
    print("ZMax mean:",zmax_anom[~np.isnan(zmax_anom.value)].mean(),len(zmax_anom),"default:",zmax_all.mean())
    #print(zmax_anom,"!! Anom 1")
    bins = np.linspace(0, 10, 50)
    plt.hist(iZAll.value, bins=bins, alpha=0.4, density=True, label='all')
    plt.hist(zmax_anom.value, bins=bins, alpha=0.4, density=True, label='anom')
    plt.legend(loc='best', fontsize=14)
    plt.yscale('log')
    plt.xlabel(r" zmax" + " [{0:latex}]".format(zmax_all.unit))
    plt.show()

    #zmax_all  = orbits_all.eccentricity()
    zmax_anom = orbits_anom.eccentricity()
    print("Ecc mean:",zmax_anom[~np.isnan(zmax_anom.value)].mean(),len(zmax_anom),"default:",zmax_all.mean())
    bins = np.linspace(0, 10, 50)
    plt.hist(iEccAll.value,  bins=bins, alpha=0.4, density=True, label='all')
    plt.hist(zmax_anom.value, bins=bins, alpha=0.4, density=True, label='anom')
    plt.legend(loc='best', fontsize=14)
    plt.yscale('log')
    plt.xlabel('Eccentricity')
    plt.show()

plotAnomalyBasic(45,processed_data_raw,testloss,galcen_clean,orbits_all)

In [ ]:
#>>>RUN: L25.4-runcell10

#Note, this will take some time to run
zmax_all = orbits_all.zmax(approximate=True)
ecc_all  = orbits_all.eccentricity()
plotAnomalyComplex(20,processed_data_raw,testloss,galcen_clean,orbits_all,ecc_all,zmax_all,iT2=100*u.Myr)

<a name='exercises_25_4'></a>     

| [Top](#section_25_0) | [Restart Section](#section_25_4) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 25.4.1</span>

We performed the analysis above using a multilayer perceptron (MLP). Consider what similarities or differences might arise if we instead used a variational autoencoder (VAE). Select ALL statements that accurately describe these machine learning methods.

A) A VAE would provide a probabilistic representation of the data, while an MLP would provide a deterministic mapping from input to output.

B) Both VAE and MLP can be used for anomaly detection, but a VAE can generate new samples similar to the training data, which an MLP cannot do.

C) A VAE would learn a continuous latent space, while an MLP would not.


**Afterwards, perform the same analysis as above with a VAE, using 4 latent dimensions.** What do you find? Note, you can check the solution to this problem to see the code that we used.

<br>